In [ ]:
import xarray as xr

In [ ]:
import schimpy
from schimpy import schism_mesh

In [ ]:
import holoviews as hv
hv.extension('bokeh')
from holoviews import opts, dim
from holoviews.operation import datashader
import warnings
warnings.filterwarnings('ignore')


## read in grid

In [ ]:
grid = schism_mesh.read_mesh('../tests/data/m1_hello_schism/hgrid.gr3')

## Fast rendering of grid

In [ ]:
trimesh = hv.TriMesh((grid.elems, grid.nodes))
# rasterize to view faster, zoom in to clarify features
img = datashader.rasterize(trimesh.edgepaths).opts(opts.Image(cmap=['darkblue'])).opts(width=800, height=400)
# spread image pixels to see mesh in a more bold style
elems_only = datashader.spread(img)

nodes_only = datashader.dynspread(datashader.rasterize(trimesh.nodes).opts(opts.Image(cmap=['blue'])), shape='circle', max_px=6)

full_mesh = (elems_only*nodes_only)

In [ ]:
import panel as pn
pn.extension()

pn.Row(full_mesh).servable(title='Mesh visualized with nodes and edge paths')#.show()

## Views of the mesh (2D projected) for each of the axes

In [ ]:
grid.nodes = grid.nodes*[1,1,-1] # need to reverse z-nodes to be negative 

In [ ]:
import numpy as np

dim_names=np.array(['length','width','depth'])

In [ ]:
def create_mesh(nodes_slice=[0,1]):
    trimeshz = hv.TriMesh((grid.elems, grid.nodes[:,nodes_slice]))#, kdims=['x','y'])
    trimeshz.nodes.redim(**dict(zip(map(str,nodes_slice),dim_names[nodes_slice])))
    # rasterize to view faster, zoom in to clarify features
    imgz = datashader.rasterize(trimeshz.edgepaths).opts(opts.Image(cmap=['darkblue'])).opts(width=800, height=400)
    # spread image pixels to see mesh in a more bold style
    elems_onlyz = datashader.spread(imgz)

    nodes_onlyz = datashader.dynspread(datashader.rasterize(trimeshz.nodes).opts(opts.Image(cmap=['blue'])), 
                                       shape='circle', max_px=6)

    full_meshz = (elems_onlyz*nodes_onlyz)
    return full_meshz

In [ ]:
hv.Layout([create_mesh([0,1]).opts(title='Top (View from z axis)',ylabel='y',xlabel='x'),
           create_mesh([0,2]).opts(title='Side (View from x axis)',ylabel='z',xlabel='y'),
           create_mesh([1,2]).opts(title='Front (View from y axis)',ylabel='z',xlabel='x')
          ]).cols(1).opts(shared_axes=False)